In [67]:
    with __import__('importnb').Notebook():
        try: from . import schema
        except: import schema
    import pathlib, nbconvert, json, pydantic, __main__
    import httpx
    import yaml
    import itertools
    import enum, typing, click, json, contextlib, io

In [38]:
    @click.group()
    def cli(): ...

In [71]:
    @cli.command()
    def strip_outputs():
        global nb
        for file in pathlib.Path().rglob('*.ipynb'):
            nb, changed = json.loads(file.read_text()), False
            for cell in nb['cells']:
                if 'outputs' in cell and cell['outputs']:
                    cell['outputs'] = []
                    changed = True
            else: changed and file.write_text(json.dumps(nb))
                
        return 0

In [72]:
    @cli.command()
    def strip_execution_count():
        global nb
        for file in pathlib.Path().rglob('*.ipynb'):
            nb, changed = json.loads(file.read_text()), False
            for cell in nb['cells']:
                if 'execution_count' in cell and cell['execution_count'] is not None:
                    cell['execution_count'] = None
                    changed = True
            else: changed and file.write_text(json.dumps(nb))
        return 0

In [74]:
    @cli.command()
    def blacken(length=88):
        import black
        blacken = lambda x: black.format_str(x, black.FileMode(line_length=length))
        for file in pathlib.Path().rglob('*.ipynb'):
            nb, changed = json.loads(file.read_text()), False
            for cell in nb['cells']:
                if cell['cell_type'] == 'cell':
                    before = cell['source']
                    try:
                        cell['source'] = blacken(''.join(before))
                        if isinstance(before, list):
                            cell['source'] = cell['source'].splitlines(True)
                        if cell['source'] != before: changed = True
                    except: ...
            else: changed and file.write_text(json.dumps(nb))
        return 0

In [81]:
    @cli.command()
    def isort(length=88):
        import isort
        for file in pathlib.Path().rglob('*.ipynb'):
            nb, changed = json.loads(file.read_text()), False
            for cell in nb['cells']:
                if cell['cell_type'] == 'cell':
                    before = cell['source']
                    try:
                        cell['source'] = isort.SortImports(file_contents=''.join(before)).output
                        if isinstance(before, list):
                            cell['source'] = cell['source'].splitlines(True)
                        if cell['source'] != before: changed = True
                    except: ...
            else: changed and file.write_text(json.dumps(nb))
        return 0